In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.docstore.document import Document  # Asegúrate de que esta línea esté incluida
from langchain_chroma import Chroma
import re


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
#carga PDF
PATH_PDF = "docs\constitucionar.pdf"
embeddings_model = OpenAIEmbeddings()

loader = PyPDFLoader(PATH_PDF)
documents = loader.load_and_split()

In [5]:
full_text = " ".join([doc.page_content for doc in documents])
full_document = Document(page_content=full_text)

In [6]:
# Dividir el texto en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,  # Tamaño del chunk aumentado
    chunk_overlap=500,  # Solapamiento del chunk aumentado
    separators=["Artículo", "Art.", "\n\n"]
)
chunks = text_splitter.split_documents([full_document])


In [7]:
def add_metadata(chunks):
    updated_chunks = []
    for chunk in chunks:
        match = re.search(r'Artículo\s*(\d+)', chunk.page_content)
        if match:
            article_number = match.group(1)
            chunk.metadata = {"article_number": article_number}
        updated_chunks.append(chunk)
    return updated_chunks

chunks = add_metadata(chunks)

In [8]:
# Verificar los chunks generados
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:")
    print(chunk.page_content)
    print("\n" + "-"*80 + "\n")

Chunk 1:
Esta publicación fue realizada por el Área de Publicacio-
nes de la Secretaría de Derechos Humanos y Pluralismo Cultural del Ministerio de Justicia y Derechos Humanos de la Nación.
Av. del Libertador 8151
Ciudad Autónoma de Buenos Airespublicacionessdh@jus.gob.arwww.derhuman.jus.gob.ar
Hecho el depósito que establece la Ley N
o 11.723
Impreso en la Argentina1a edición: abril de 2016
© Secretaría de Derechos Humanos y Pluralismo CulturalArgentina. Constitución
   Constitución de la Nación Argentina. - 1a ed . - Ciu-dad Autónoma de Buenos Aires : Ministerio de Justicia y Derechos Humanos de la Nación. Secretaría de Dere-chos Humanos y Pluralismo Cultural, 2016.   68 p. ; 13 x 9 cm. - (De bolsillo)
   ISBN 978-987-4017-10-9   1. Constitución de la Nación Argentina.    
  CDD 342.023 Presidente de la Nación
Ing. Mauricio Macri 
 
Ministro de Justicia y Derechos Humanos
Dr. Germán Garavano 
Secretario de Derechos Humanos 
y Pluralismo Cultural 
Lic. Claudio Avruj La Secretaría de D

In [9]:
vectorstore = Chroma.from_documents(
    documents=chunks,  # Usar los chunks generados
    embedding=embeddings_model,
    persist_directory="docs/Chroma"
)